In [1]:
import numpy as np
import cv2
from keras.layers import Input, Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D, UpSampling2D, AveragePooling2D
from keras.layers import BatchNormalization, Activation, concatenate, Lambda, add, Dropout
from keras.models import Model
import keras.backend as K
from keras import optimizers
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
img = cv2.imread('../../render/210324/exr/img-0-diffuse.exr', -1)
img.shape

(512, 512, 3)

In [3]:
img[300:305, 300:305, :]

array([[[29.796875, 29.796875, 29.796875],
        [29.15625 , 29.15625 , 29.15625 ],
        [28.609375, 28.609375, 28.609375],
        [28.078125, 28.078125, 28.078125],
        [27.359375, 27.359375, 27.359375]],

       [[29.71875 , 29.71875 , 29.71875 ],
        [29.265625, 29.265625, 29.265625],
        [28.53125 , 28.53125 , 28.53125 ],
        [27.921875, 27.921875, 27.921875],
        [27.53125 , 27.53125 , 27.53125 ]],

       [[29.765625, 29.765625, 29.765625],
        [29.265625, 29.265625, 29.265625],
        [28.59375 , 28.59375 , 28.59375 ],
        [27.796875, 27.796875, 27.796875],
        [27.21875 , 27.21875 , 27.21875 ]],

       [[30.      , 30.      , 30.      ],
        [29.15625 , 29.15625 , 29.15625 ],
        [28.453125, 28.453125, 28.453125],
        [27.953125, 27.953125, 27.953125],
        [28.171875, 28.171875, 28.171875]],

       [[29.6875  , 29.6875  , 29.6875  ],
        [29.      , 29.      , 29.      ],
        [28.234375, 28.234375, 28.234375],
   

In [4]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(512,512,3))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def encode_block(x, ch):
    def base_block(x):
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        # x = Dropout(rate=drop_rate)(x)
        x = Conv2D(ch, (3, 3), padding='same')(x)
        return x

    x = base_block(x)
    x = base_block(x)
    return x

def decode_block(x, c, ch):
    ch = ch
    def base_block(x):
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        # x = Dropout(rate=drop_rate)(x)
        x = Conv2DTranspose(ch, (3, 3), padding='same')(x)
        return x

    x = Conv2DTranspose(ch, (3, 3), padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = concatenate([x, c])

    x = base_block(x)
    x = base_block(x)
    return x

batch_shape = (512, 512)
ch_num = 3

input_batch = Input(shape=(*batch_shape, ch_num))
e0 = Conv2D(8, (1, 1), padding='same')(input_batch)
e0 = Activation('relu')(e0)

e0 = encode_block(e0, 16)

e1 = AveragePooling2D((2, 2))(e0)
e1 = encode_block(e1, 32)

e2 = AveragePooling2D((2, 2))(e1)
e2 = encode_block(e2, 64)

e3 = AveragePooling2D((2, 2))(e2)
e3 = encode_block(e3, 128)

d2 = decode_block(e3, e2, 64)
d1 = decode_block(d2, e1, 32)
d0 = decode_block(d1, e0, 16)

output_batch = Conv2D(ch_num, (1, 1), padding='same')(d0)

model = Model(input_batch, output_batch)

In [5]:
loss_model = Model(vgg_model.inputs, vgg_model.outputs)

In [6]:
loss_model.compile(optimizer='adam', loss='mse')

In [7]:
batch = img[None, ...]
feature = loss_model.predict(batch)

In [9]:
feature.shape

(1, 16, 16, 512)

In [10]:
selected_layers = [1,2,9,10,17,18]
selected_outputs = [vgg_model.layers[i].output for i in selected_layers]
multi_loss_model = Model(vgg_model.inputs, selected_outputs)
multi_loss_model.compile(optimizer='adam', loss='mse')

In [12]:
multi_feature = multi_loss_model.predict(batch)

In [14]:
len(multi_feature)

6

In [15]:
for i in range(len(multi_feature)):
    print(multi_feature[i].shape)

(1, 512, 512, 64)
(1, 512, 512, 64)
(1, 128, 128, 256)
(1, 64, 64, 256)
(1, 32, 32, 512)
(1, 16, 16, 512)
